# Sentiment Analysis with an RNN

In this notebook, you'll implement a recurrent neural network that performs sentiment analysis. 
>Using an RNN rather than a strictly feedforward network is more accurate since we can include information about the *sequence* of words. 

Here we'll use a dataset of movie reviews, accompanied by sentiment labels: positive or negative.

<img src="assets/reviews_ex.png" width=40%>

### Network Architecture

The architecture for this network is shown below.

<img src="assets/network_diagram.png" width=40%>

>**First, we'll pass in words to an embedding layer.** We need an embedding layer because we have tens of thousands of words, so we'll need a more efficient representation for our input data than one-hot encoded vectors. You should have seen this before from the Word2Vec lesson. You can actually train an embedding with the Skip-gram Word2Vec model and use those embeddings as input, here. However, it's good enough to just have an embedding layer and let the network learn a different embedding table on its own. *In this case, the embedding layer is for dimensionality reduction, rather than for learning semantic representations.*

>**After input words are passed to an embedding layer, the new embeddings will be passed to LSTM cells.** The LSTM cells will add *recurrent* connections to the network and give us the ability to include information about the *sequence* of words in the movie review data. 

>**Finally, the LSTM outputs will go to a sigmoid output layer.** We're using a sigmoid function because positive and negative = 1 and 0, respectively, and a sigmoid will output predicted, sentiment values between 0-1. 

We don't care about the sigmoid outputs except for the **very last one**; we can ignore the rest. We'll calculate the loss by comparing the output at the last time step and the training label (pos or neg).

---
### Load in and visualize the data

In [1]:
import numpy as np

# read data from text files
with open('data/reviews.txt', 'r') as f:
    reviews = f.read()
with open('data/labels.txt', 'r') as f:
    labels = f.read()

In [2]:
print(reviews[:2000])
print()
print(labels[:20])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

## Data pre-processing

The first step when building a neural network model is getting your data into the proper form to feed into the network. Since we're using embedding layers, we'll need to encode each word with an integer. We'll also want to clean it up a bit.

You can see an example of the reviews data above. Here are the processing steps, we'll want to take:
>* We'll want to get rid of periods and extraneous punctuation.
* Also, you might notice that the reviews are delimited with newline characters `\n`. To deal with those, I'm going to split the text into each review using `\n` as the delimiter. 
* Then I can combined all the reviews back together into one big string.

First, let's remove all punctuation. Then get all the text without the newlines and split it into individual words.

In [3]:
single_quote_contractions = { 
"he ain't got": "he does not have",
"she ain't got": "she does not have",
"it ain't got": "it does not have",
"ain't got": "do not have",
"i ain't": "i am not",
"she ain't": "she is not",
"it ain't": "it is not",
"ain't": "are not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had", # kinda wrong
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's got": "he has",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is", # kinda wrong
"i'd": "i had", # kinda wrong
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it had", # kinda wrong
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's got": "it has",
"it's": "it is", 
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had", # kinda wrong
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's got": "she has",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that had", # kinda wrong
"that'd've": "that would have",
"that's got": "that has",
"that's": "that is",
"there'd": "there had", # kinda wrong
"there'd've": "there would have",
"there's got": "there has",
"there's": "there is",
"they'd": "they had",  # kinda wrong
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had",  # kinda wrong
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's got": "what has",
"what's": "what is",
"what've": "what have",
"when's got": "when has",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's got": "where has",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's got": "who has",
"who's": "who is",
"who've": "who have",
"why's got": "why has",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had",  # kinda wrong
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

space_contractions = {contraction.replace("'", " "): expanded for contraction, expanded in single_quote_contractions.items()}
contractions = {**single_quote_contractions, **space_contractions}

In [26]:
from string import punctuation
import re

def preprocess(sometext):
    sometext = sometext.lower() # lowercase, standardize
    sometext = sometext.replace(' br ','')
    for contraction, replacement in single_quote_contractions.items():
        sometext = sometext.replace(contraction, replacement)

    sometext = ''.join([c for c in sometext if c not in punctuation or c == '\n'])
    sometext = re.sub(r' +', ' ', sometext)

    for contraction, replacement in space_contractions.items():
        sometext = sometext.replace(contraction, replacement)
    return sometext

# get rid of punctuation
reviews = preprocess(reviews)

In [5]:
# split by new lines and spaces
reviews_split = reviews.split('\n')
reviews = ' '.join(reviews_split)

# create a list of words
words = reviews.split()

### Encoding the words

The embedding lookup requires that we pass in integers to our network. The easiest way to do this is to create dictionaries that map the words in the vocabulary to integers. Then we can convert each of our reviews into integers so they can be passed into the network.

> **Exercise:** Now you're going to encode the words with integers. Build a dictionary that maps words to integers. Later we're going to pad our input vectors with zeros, so make sure the integers **start at 1, not 0**.
> Also, convert the reviews to integers and store the reviews in a new list called `reviews_ints`. 

In [6]:
# feel free to use this import 
from collections import Counter

## Build a dictionary that maps words to integers
word_counts = Counter(words)
vocab = [word for (word,count) in word_counts.most_common(n=len(words))]
int_to_vocab = {index + 1:vocab[index] for index in range(len(vocab))}
vocab_to_int = {word:index for index, word in int_to_vocab.items()}

def tokenize(review):
    return [vocab_to_int[word] for word in review.split()]

## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
reviews_ints = [tokenize(review) for review in reviews_split]

print([int_to_vocab[key] for key in int_to_vocab.keys()][:10])
print([vocab_to_int[key] for key in vocab_to_int.keys()][:10])

['the', 'and', 'a', 'of', 'is', 'to', 'it', 'in', 'i', 'this']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


**Test your code**

As a text that you've implemented the dictionary correctly, print out the number of unique words in your vocabulary and the contents of the first, tokenized review.

In [7]:
# stats about vocabulary
print('Unique words: ', len((vocab_to_int)))  # should ~ 74000+
print()

# print tokens in first review
print('Tokenized review: \n', reviews_ints[:2])

Unique words:  80054

Tokenized review: 
 [[21865, 291, 5, 3, 1033, 201, 7, 2136, 30, 1, 174, 57, 14, 48, 79, 5887, 42, 403, 108, 136, 14, 5277, 59, 149, 8, 1, 5037, 5945, 461, 69, 6, 251, 11, 21865, 291, 24, 2175, 5, 73, 2408, 6, 602, 72, 5, 5277, 1, 49416, 6, 2037, 10401, 1, 5888, 1791, 34, 71, 62, 198, 138, 63, 1173, 5277, 20629, 1, 39531, 4, 1, 212, 1174, 29, 3035, 69, 4, 1, 19518, 9, 662, 2, 63, 1791, 50, 9, 217, 1, 362, 8, 60, 3, 1514, 3751, 767, 6, 3495, 184, 1, 3116, 9, 1188, 13987, 30, 291, 3, 330, 323, 2914, 9, 98, 124, 6, 9004, 28, 4, 126, 5277, 1514, 2338, 6, 21865, 291, 9, 510, 11, 106, 1433, 4, 59, 524, 99, 11, 21865, 291, 5, 218, 4164, 46, 3, 2214, 11, 7, 5, 12], [111, 4, 3, 123, 34, 43, 7627, 1375, 15, 3, 4207, 698, 44, 16, 3, 605, 155, 11, 5, 3, 1260, 445, 4, 1711, 201, 3, 10889, 7514, 283, 5, 649, 81, 33, 2112, 1068, 2997, 32, 1, 875, 49417, 4, 7, 5, 5607, 449, 7, 6390, 1711, 1, 212, 57, 16, 56, 779, 1280, 824, 219, 7, 41, 94, 121, 1451, 58, 142, 35, 1, 940, 167, 27, 

### Encoding the labels

Our labels are "positive" or "negative". To use these labels in our network, we need to convert them to 0 and 1.

> **Exercise:** Convert labels from `positive` and `negative` to 1 and 0, respectively, and place those in a new list, `encoded_labels`.

In [8]:
# 1=positive, 0=negative label conversion
label_to_int = {'positive':1, 'negative': 0}
encoded_labels = [label_to_int[label] for label in labels.split('\n') if not label == '']

### Removing Outliers

As an additional pre-processing step, we want to make sure that our reviews are in good shape for standard processing. That is, our network will expect a standard input text size, and so, we'll want to shape our reviews into a specific length. We'll approach this task in two main steps:

1. Getting rid of extremely long or short reviews; the outliers
2. Padding/truncating the remaining data so that we have reviews of the same length.

<img src="assets/outliers_padding_ex.png" width=40%>

Before we pad our review text, we should check for reviews of extremely short or long lengths; outliers that may mess with our training.

In [9]:
# outlier review stats
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 1
Maximum review length: 2492


Okay, a couple issues here. We seem to have one review with zero length. And, the maximum review length is way too many steps for our RNN. We'll have to remove any super short reviews and truncate super long reviews. This removes outliers and should allow our model to train more efficiently.

> **Exercise:** First, remove *any* reviews with zero length from the `reviews_ints` list and their corresponding label in `encoded_labels`.

In [10]:
print('Number of reviews before removing outliers: ', len(reviews_ints))

## remove any reviews/labels with zero length from the reviews_ints list.
non_zero_indices = [index for index, review in enumerate(reviews_ints) if len(review) > 0]
    
reviews_ints = [reviews_ints[index] for index in non_zero_indices]
encoded_labels = [encoded_labels[index] for index in non_zero_indices]

print('Number of reviews after removing outliers: ', len(reviews_ints))

Number of reviews before removing outliers:  25001
Number of reviews after removing outliers:  25000


---
## Padding sequences

To deal with both short and very long reviews, we'll pad or truncate all our reviews to a specific length. For reviews shorter than some `seq_length`, we'll pad with 0s. For reviews longer than `seq_length`, we can truncate them to the first `seq_length` words. A good `seq_length`, in this case, is 200.

> **Exercise:** Define a function that returns an array `features` that contains the padded data, of a standard size, that we'll pass to the network. 
* The data should come from `review_ints`, since we want to feed integers to the network. 
* Each row should be `seq_length` elements long. 
* For reviews shorter than `seq_length` words, **left pad** with 0s. That is, if the review is `['best', 'movie', 'ever']`, `[117, 18, 128]` as integers, the row will look like `[0, 0, 0, ..., 0, 117, 18, 128]`. 
* For reviews longer than `seq_length`, use only the first `seq_length` words as the feature vector.

As a small example, if the `seq_length=10` and an input review is: 
```
[117, 18, 128]
```
The resultant, padded sequence should be: 

```
[0, 0, 0, 0, 0, 0, 0, 117, 18, 128]
```

**Your final `features` array should be a 2D array, with as many rows as there are reviews, and as many columns as the specified `seq_length`.**

This isn't trivial and there are a bunch of ways to do this. But, if you're going to be building your own deep learning networks, you're going to have to get used to preparing your data.

In [11]:
def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    ## implement function
    all_zero_sequence = np.zeros(seq_length, dtype='int')
    features = []
    
    for review in reviews_ints:
        length = len(review)
        if length >= seq_length:
            features.append(review[:seq_length])
        else:
            feature = all_zero_sequence.copy()
            feature[-length:] = review
            features.append(feature)
    
    return np.array(features)

In [12]:
# Test your implementation!

seq_length = 200

features = pad_features(reviews_ints, seq_length=seq_length)

## test statements - do not change - ##
assert len(features)==len(reviews_ints), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 10 values of the first 30 batches 
print(features[-50:,-12:])

[[  656    64     5     3  1107   492    11     5   165     6    27 14814]
 [  229     4  1098  5227   919     6  4647  5708   449     1   402     2]
 [ 1473   470    88   548  1475  9571   204     5     3    19     6  5892]
 [13614    14     1  8961   216    64 15657     8     1    17   477  5676]
 [   38    13  1042     9    13   553   113   222   311     2   113   836]
 [  226    16   176  1267   494     2     3   110    11    13    12    38]
 [    7     6   207    44    21   383     3   237   197    15    59  1560]
 [ 1961     7     2     1  1859   698     3  3222  3148    15    10   216]
 [ 1278  1397     9    67   360    10    17    15     3  1839  2362  2608]
 [    7    81     3   673 10920    38   134    59 14946   427     4  3642]
 [   20    23    12   107     1 16015   845   325     1   257    21    60]
 [80020     6   444   561   301 22898    69     9   390   220    15   158]
 [    6    27   478     6    62  1611   100   207     6   108    37 20142]
 [   98   626    42    18

## Training, Validation, Test

With our data in nice shape, we'll split it into training, validation, and test sets.

> **Exercise:** Create the training, validation, and test sets. 
* You'll need to create sets for the features and the labels, `train_x` and `train_y`, for example. 
* Define a split fraction, `split_frac` as the fraction of data to **keep** in the training set. Usually this is set to 0.8 or 0.9. 
* Whatever data is left will be split in half to create the validation and *testing* data.

In [13]:
import math
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)
n_features = len(features)
shuffled_indices = np.random.permutation(range(n_features))

train_validation_split = math.floor(n_features * split_frac)
train_indices = shuffled_indices[:train_validation_split]

validation_test_split = train_validation_split + math.floor(n_features * (1 - split_frac) * 0.5) + 1
validation_indices = shuffled_indices[train_validation_split:validation_test_split]

test_indices = shuffled_indices[validation_test_split:]

train_x = np.array([features[index] for index in train_indices])
train_y = np.array([encoded_labels[index] for index in train_indices])

validation_x = np.array([features[index] for index in validation_indices])
validation_y = np.array([encoded_labels[index] for index in validation_indices])

test_x = np.array([features[index] for index in test_indices])
test_y = np.array([encoded_labels[index] for index in test_indices])

## print out the shapes of your resultant feature data
print(f'Train set: \t\t {train_x.shape}')
print(f'Validation set: \t {validation_x.shape}')
print(f'Test set: \t\t {test_x.shape}')

Train set: 		 (20000, 200)
Validation set: 	 (2500, 200)
Test set: 		 (2500, 200)


**Check your work**

With train, validation, and test fractions equal to 0.8, 0.1, 0.1, respectively, the final, feature data shapes should look like:
```
                    Feature Shapes:
Train set: 		 (20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		  (2500, 200)
```

---
## DataLoaders and Batching

After creating training, test, and validation data, we can create DataLoaders for this data by following two steps:
1. Create a known format for accessing our data, using [TensorDataset](https://pytorch.org/docs/stable/data.html#) which takes in an input set of data and a target set of data with the same first dimension, and creates a dataset.
2. Create DataLoaders and batch our training, validation, and test Tensor datasets.

```
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
train_loader = DataLoader(train_data, batch_size=batch_size)
```

This is an alternative to creating a generator function for batching our data into full batches.

In [14]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(validation_x), torch.from_numpy(validation_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 25

# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [15]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([25, 200])
Sample input: 
 tensor([[     0,      0,      0,  ...,     36,     11,      8],
        [     7,      5,     52,  ...,  19867,   6452,   3407],
        [     0,      0,      0,  ...,    446,    215,  13862],
        ...,
        [     0,      0,      0,  ...,     94,    186,     58],
        [     0,      0,      0,  ...,      3,    163,     62],
        [  5954,      3,   9368,  ...,     10,    154,     18]])

Sample label size:  torch.Size([25])
Sample label: 
 tensor([ 1,  1,  0,  0,  1,  0,  0,  1,  0,  1,  0,  1,  1,  0,
         0,  0,  1,  0,  1,  1,  1,  1,  1,  1,  1])


---
# Sentiment Network with PyTorch

Below is where you'll define the network.

<img src="assets/network_diagram.png" width=40%>

The layers are as follows:
1. An [embedding layer](https://pytorch.org/docs/stable/nn.html#embedding) that converts our word tokens (integers) into embeddings of a specific size.
2. An [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) defined by a hidden_state size and number of layers
3. A fully-connected output layer that maps the LSTM layer outputs to a desired output_size
4. A sigmoid activation layer which turns all outputs into a value 0-1; return **only the last sigmoid output** as the output of this network.

### The Embedding Layer

We need to add an [embedding layer](https://pytorch.org/docs/stable/nn.html#embedding) because there are 74000+ words in our vocabulary. It is massively inefficient to one-hot encode that many classes. So, instead of one-hot encoding, we can have an embedding layer and use that layer as a lookup table. You could train an embedding layer using Word2Vec, then load it here. But, it's fine to just make a new layer, using it for only dimensionality reduction, and let the network learn the weights.


### The LSTM Layer(s)

We'll create an [LSTM](https://pytorch.org/docs/stable/nn.html#lstm) to use in our recurrent network, which takes in an input_size, a hidden_dim, a number of layers, a dropout probability (for dropout between multiple layers), and a batch_first parameter.

Most of the time, you're network will have better performance with more layers; between 2-3. Adding more layers allows the network to learn really complex relationships. 

> **Exercise:** Complete the `__init__`, `forward`, and `init_hidden` functions for the SentimentRNN model class.

Note: `init_hidden` should initialize the hidden and cell state of an lstm layer to all zeros, and move those state to GPU, if available.

In [16]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [17]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # define all layers
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, dropout=drop_prob, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        
        batch_size = x.size(0)
            
        x = self.embed(x)
        x, hidden = self.lstm(x, hidden)
        x = x.contiguous().view(-1, self.hidden_dim)

        x = self.fc(x)
        sig_out = self.sigmoid(x)
        
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1]
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        

## Instantiate the network

Here, we'll instantiate the network. First up, defining the hyperparameters.

* `vocab_size`: Size of our vocabulary or the range of values for our input, word tokens.
* `output_size`: Size of our desired output; the number of class scores we want to output (pos/neg).
* `embedding_dim`: Number of columns in the embedding lookup table; size of our embeddings.
* `hidden_dim`: Number of units in the hidden layers of our LSTM cells. Usually larger is better performance wise. Common values are 128, 256, 512, etc.
* `n_layers`: Number of LSTM layers in the network. Typically between 1-3

> **Exercise:** Define the model  hyperparameters.


In [18]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab) + 1
output_size = 1
embedding_dim = 200
hidden_dim = 256
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embed): Embedding(80055, 200)
  (lstm): LSTM(200, 256, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


---
## Training

Below is the typical training code. If you want to do this yourself, feel free to delete all this code and implement it yourself. You can also add code to save a model by name.

>We'll also be using a new kind of cross entropy loss, which is designed to work with a single Sigmoid output. [BCELoss](https://pytorch.org/docs/stable/nn.html#bceloss), or **Binary Cross Entropy Loss**, applies cross entropy loss to a single value between 0 and 1.

We also have some data and training hyparameters:

* `lr`: Learning rate for our optimizer.
* `epochs`: Number of times to iterate through the training dataset.
* `clip`: The maximum gradient value to clip at (to prevent exploding gradients).

In [19]:
# loss and optimization functions
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)


In [20]:
# training params

epochs = 4 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/4... Step: 100... Loss: 0.636947... Val Loss: 0.677768
Epoch: 1/4... Step: 200... Loss: 0.706266... Val Loss: 0.664541
Epoch: 1/4... Step: 300... Loss: 0.814421... Val Loss: 0.832974
Epoch: 1/4... Step: 400... Loss: 0.690579... Val Loss: 0.767746
Epoch: 1/4... Step: 500... Loss: 0.630785... Val Loss: 0.793348
Epoch: 1/4... Step: 600... Loss: 0.652771... Val Loss: 0.667695
Epoch: 1/4... Step: 700... Loss: 0.547784... Val Loss: 0.606117
Epoch: 1/4... Step: 800... Loss: 0.546831... Val Loss: 0.577732
Epoch: 2/4... Step: 900... Loss: 0.433835... Val Loss: 0.568638
Epoch: 2/4... Step: 1000... Loss: 0.500424... Val Loss: 0.530898
Epoch: 2/4... Step: 1100... Loss: 0.418165... Val Loss: 0.524988
Epoch: 2/4... Step: 1200... Loss: 0.323654... Val Loss: 0.474581
Epoch: 2/4... Step: 1300... Loss: 0.434524... Val Loss: 0.495485
Epoch: 2/4... Step: 1400... Loss: 0.575772... Val Loss: 0.450351
Epoch: 2/4... Step: 1500... Loss: 0.378593... Val Loss: 0.447837
Epoch: 2/4... Step: 1600... Loss: 

---
## Testing

There are a few ways to test your network.

* **Test data performance:** First, we'll see how our trained model performs on all of our defined test_data, above. We'll calculate the average loss and accuracy over the test data.

* **Inference on user-generated data:** Second, we'll see if we can input just one example review at a time (without a label), and see what the trained model predicts. Looking at new, user input data like this, and predicting an output label, is called **inference**.

In [21]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.374
Test accuracy: 0.836


### Inference on a test review

You can change this test_review to any text that you want. Read it and think: is it pos or neg? Then see if your model predicts correctly!
    
> **Exercise:** Write a `predict` function that takes in a trained net, a plain text_review, and a sequence length, and prints out a custom statement for a positive or negative review!
* You can use any functions that you've already defined or define any helper functions you want to complete `predict`, but it should just take in a trained net, a text review, and a sequence length.


In [72]:
# negative test review
test_reviews = [
    'The worst movie I have seen; acting was terrible and I want my money back. This movie had bad acting and the dialogue was slow.',
    'The best movie I have seen; acting was magnificent and I would pay twice the money. This movie had great acting and the dialogue was genius.',
    'The most average movie I have seen; acting was neither good nor bad and I do not regret seeing it. This movie hat interesting dialogue, but lacked creativity.',
    'Wow. Just wow!',
    'Simply brilliant!',
    'I am not going to lie: this movie was pretty dope.',
    'I am utterly confused. This was one of the weirdest movies I have ever seen. Not that it was not good, but just so strange. Would see it again.'
]


In [69]:
def predict(net, test_review, sequence_length=200):
    ''' Prints out whether a give review is predicted to be 
        positive or negative in sentiment, using a trained model.
        
        params:
        net - A trained net 
        test_review - a review made of normal text and punctuation
        sequence_length - the padded length of a review
        '''
    net.eval()
    x = preprocess(test_review)
    x = tokenize(x)
    x = pad_features([x], seq_length=sequence_length)
    x = torch.from_numpy(x)
    if(train_on_gpu):
        x = x.cuda()
        
    h = net.init_hidden(1)
    
    prediction, _ = net(x, h)
    prediction = prediction.item()
    confidence = np.abs(0.5 - prediction) * 200
    sentiment = 'positive' if np.round(prediction) == 1 else 'negative'
    print(f'I am {confidence:.1f}% confident that this is a {sentiment} review:\n\t"{test_review}"')
    return prediction

In [73]:
# call function
# try negative and positive reviews!
for review in test_reviews:
    predict(net, review, seq_length)

I am 97.7% confident that this is a negative review:
	"The worst movie I have seen; acting was terrible and I want my money back. This movie had bad acting and the dialogue was slow."
I am 74.3% confident that this is a positive review:
	"The best movie I have seen; acting was magnificent and I would pay twice the money. This movie had great acting and the dialogue was genius."
I am 84.0% confident that this is a negative review:
	"The most average movie I have seen; acting was neither good nor bad and I do not regret seeing it. This movie hat interesting dialogue, but lacked creativity."
I am 76.5% confident that this is a negative review:
	"Wow. Just wow!"
I am 42.3% confident that this is a positive review:
	"Simply brilliant!"
I am 86.3% confident that this is a negative review:
	"I am not going to lie: this movie was pretty dope."
I am 95.9% confident that this is a negative review:
	"I am utterly confused. This was one of the weirdest movies I have ever seen. Not that it was not 

### Try out test_reviews of your own!

Now that you have a trained model and a predict function, you can pass in _any_ kind of text and this model will predict whether the text has a positive or negative sentiment. Push this model to its limits and try to find what words it associates with positive or negative.

Later, you'll learn how to deploy a model like this to a production environment so that it can respond to any kind of user data put into a web app!